Uninstall existing versions and install specific versions of gym and related packages

In [ ]:
!pip uninstall -y gym gym-super-mario-bros
!pip install gym==0.26.2 gym-notices gym-super-mario-bros==7.4.0
!pip install stable-baselines3 gym_super_mario_bros nes_py

Found existing installation: gym 0.26.2
Uninstalling gym-0.26.2:
  Successfully uninstalled gym-0.26.2
Found existing installation: gym-super-mario-bros 7.4.0
Uninstalling gym-super-mario-bros-7.4.0:
  Successfully uninstalled gym-super-mario-bros-7.4.0
  Using cached gym-0.26.2-py3-none-any.whl
  Using cached gym_super_mario_bros-7.4.0-py3-none-any.whl.metadata (10 kB)
Using cached gym_super_mario_bros-7.4.0-py3-none-any.whl (199 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.0.9 requires gym<=0.25.2, but you have gym 0.26.2 which is incompatible.


In [ ]:
# Importiere grundlegende Bibliotheken
import gym
import gym_super_mario_bros
from gym.wrappers import FrameStack, GrayScaleObservation
from gym.spaces import Box
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT, COMPLEX_MOVEMENT

# Importiere Bibliotheken für numerische Berechnungen und Deep Learning
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


# Importiere weitere Hilfsbibliotheken
import random
from collections import deque
import matplotlib.pyplot as plt
import torchvision.transforms as T
import time

# Importiere Bibliotheken für die Interaktion mit Google Colab
from google.colab import files


Neural network definition

In [ ]:
#Dueling DQN Neural Network
class DuelingDQN(nn.Module):
    def __init__(self, input_shape, num_actions):
        super(DuelingDQN, self).__init__()
        self.conv1 = nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)

        # Compute the output size after convolutional layers
        def conv2d_size_out(size, kernel_size, stride):
            return (size - (kernel_size - 1) - 1) // stride + 1

        conv_h = conv2d_size_out(conv2d_size_out(conv2d_size_out(input_shape[1], 8, 4), 4, 2), 3, 1)
        conv_w = conv2d_size_out(conv2d_size_out(conv2d_size_out(input_shape[2], 8, 4), 4, 2), 3, 1)
        linear_input_size = conv_w * conv_h * 64

        self.fc1 = nn.Linear(linear_input_size, 512)

        # Define the advantage and value streams
        self.advantage_stream = nn.Linear(512, num_actions)
        self.value_stream = nn.Linear(512, 1)

    def forward(self, x):
        #x = x / 255.0  # Normalize input
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = x.view(x.size(0), -1)  # flatten
        x = torch.relu(self.fc1(x))

        # Separate into advantage and value streams
        advantage = self.advantage_stream(x)
        value = self.value_stream(x)

        # Combine advantage and value to get final Q values
        q_values = value + (advantage - advantage.mean(dim=1, keepdim=True))
        return q_values

# class DuelingDQN(nn.Module):
#     def __init__(self, input_shape, num_actions):
#         super(DuelingDQN, self).__init__()
#         self.conv1 = nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4)
#         self.bn1 = nn.BatchNorm2d(32)  # Add BatchNorm after conv1

#         self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
#         self.bn2 = nn.BatchNorm2d(64)  # Add BatchNorm after conv2

#         self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
#         self.bn3 = nn.BatchNorm2d(64)  # Add BatchNorm after conv3

#         # Compute the output size after convolutional layers
#         def conv2d_size_out(size, kernel_size, stride):
#             return (size - (kernel_size - 1) - 1) // stride + 1

#         conv_h = conv2d_size_out(conv2d_size_out(conv2d_size_out(input_shape[1], 8, 4), 4, 2), 3, 1)
#         conv_w = conv2d_size_out(conv2d_size_out(conv2d_size_out(input_shape[2], 8, 4), 4, 2), 3, 1)
#         linear_input_size = conv_w * conv_h * 64

#         self.fc1 = nn.Linear(linear_input_size, 512)

#         # Define the advantage and value streams
#         self.advantage_stream = nn.Linear(512, num_actions)
#         self.value_stream = nn.Linear(512, 1)

#     def forward(self, x):
#         x = x / 255.0  # Normalize input to [0, 1]

#         # Apply convolutional layers with BatchNorm and ReLU activations
#         x = F.relu(self.bn1(self.conv1(x)))
#         x = F.relu(self.bn2(self.conv2(x)))
#         x = F.relu(self.bn3(self.conv3(x)))

#         x = x.view(x.size(0), -1)  # Flatten the tensor

#         x = F.relu(self.fc1(x))

#         # Separate into advantage and value streams
#         advantage = self.advantage_stream(x)
#         value = self.value_stream(x)

#         # Combine advantage and value to get final Q-values
#         q_values = value + (advantage - advantage.mean(dim=1, keepdim=True))
#         return q_values


Epsilon greedy

In [ ]:
# Select action
def select_action(state, epsilon):
    if random.random() < epsilon:
        action = env.action_space.sample()
        return action
    else:
        with torch.no_grad():
            return policy_net(state).argmax().item()

Preprocess wrappers

In [ ]:
# Define Wrappers
class SkipFrame(gym.Wrapper):
    def __init__(self, env, skip):
        """Return only every `skip`-th frame"""
        super().__init__(env)
        self._skip = skip

    def step(self, action):
        """Repeat action, and sum reward"""
        total_reward = 0.0
        for i in range(self._skip):
            obs, reward, done, truncated, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return obs, total_reward, done, truncated, info

class GrayScaleObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        obs_shape = self.observation_space.shape[:2]
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

    def permute_orientation(self, observation):
        observation = np.transpose(observation, (2, 0, 1))
        observation = torch.tensor(observation.copy(), dtype=torch.float)
        return observation

    def observation(self, observation):
        observation = self.permute_orientation(observation)
        transform = T.Grayscale()
        observation = transform(observation)
        return observation

class ResizeObservation(gym.ObservationWrapper):
    def __init__(self, env, shape):
        super().__init__(env)
        if isinstance(shape, int):
            self.shape = (shape, shape)
        else:
            self.shape = tuple(shape)

        obs_shape = self.shape + self.observation_space.shape[2:]
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

    def observation(self, observation):
        transforms = T.Compose(
            [T.Resize(self.shape, antialias=True), T.Normalize(0, 255)]
        )
        observation = transforms(observation).squeeze(0)
        return observation

Environment configuration + hyperparameters

In [ ]:
# Configure environment

#env = gym_super_mario_bros.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode="none")
#env = gym_super_mario_bros.make('SuperMarioBrosRandomStages-v0', apply_api_compatibility=True, render_mode="none")
#env = gym_super_mario_bros.make('SuperMarioBrosRandomStages-v0', stages=['1-1'], apply_api_compatibility=True, render_mode="none")
env = gym_super_mario_bros.make('SuperMarioBrosRandomStages-v0', stages=['1-4'], apply_api_compatibility=True, render_mode="none")
env = JoypadSpace(env, SIMPLE_MOVEMENT)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Apply Wrappers to environment
env = SkipFrame(env, skip=4)
env = GrayScaleObservation(env)
env = ResizeObservation(env, shape=84)
if gym.__version__ < '0.26':
    env = FrameStack(env, num_stack=4, new_step_api=True)
else:
    env = FrameStack(env, num_stack=4)

# Hyperparameters
state_space = env.observation_space.shape  # (4, 84, 84)
action_space = env.action_space.n
learning_rate = 0.00015
gamma = 0.99

epsilon = 1.0
epsilon_min = 0.02
epsilon_decay = 0.99997
batch_size = 32
target_update = 20
replayBuffer_size = 300000
num_episodes = 50000
start_learning = 0
save_weights = 500
step_count = 0


# Moving average array
moving_average = []

# Replay Buffer
replayBuffer = deque(maxlen=replayBuffer_size)



# Instantiate Q-networks
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
policy_net = DuelingDQN((4, 84, 84), action_space).to(device)  # Adjusted input shape to (4, 84, 84)
target_net = DuelingDQN((4, 84, 84), action_space).to(device)  # Adjusted input shape to (4, 84, 84)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.Adam(policy_net.parameters(), lr=learning_rate)
#loss_fn = nn.MSELoss()
loss_fn = nn.SmoothL1Loss()

target_net.load_state_dict(policy_net.state_dict())

# Training loop with evaluation
training_rewards = []
evaluation_rewards = []
moving_average_training = []
moving_average_evaluation = []
min_training_rewards = []
max_training_rewards = []
min_moving_average_training = []
max_moving_average_training = []
min_moving_average_evaluation = []
max_moving_average_evaluation = []

start_episode = 23000
step_count = 1379678
epsilon = 0.02


/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment SuperMarioBrosRandomStages-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:627: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.frames_per_second']
  logger.warn(


cuda


In [ ]:
#--------------------------Additional features-------------------------------

# Reward adjustment parameters - Currently not in use
# time_penalty = -1  # Time penalty for taking too long in the environment
# position_penalty = -5  # Position penalty for moving backwards or losing ground

# These parameters are commented out because we're experimenting without penalties.
# If you want to reintroduce them, simply uncomment the lines above.

# Reward adjustment logic - Currently commented out for testing purposes
# Uncomment if you want to penalize standing still or time spent.

# if next_info['time'] < time_last:
#     reward += time_penalty  # Apply time penalty if agent takes too long
#
# if next_info['x_pos'] == x_pos_last:
#     reward += position_penalty  # Apply penalty for not moving forward
#     standing_still_counter += 1  # Count how long agent stands still
# else:
#     standing_still_counter = 0  # Reset the counter if the agent moves
#
# if standing_still_counter >= 4:
#     reward += -10  # Additional penalty for standing still too long
#
# x_pos_last = next_info['x_pos']  # Update last known position
# time_last = next_info['time']  # Update last known time

# This logic is commented out because we're testing the behavior without time or position penalties.
# To re-enable, simply uncomment the relevant lines above.


Converts state to a contiguous PyTorch tensor, adds a batch dimension, and moves it to the specified device for neural network processing.


In [ ]:
# Preprocess state
def preprocess_state(state):
    state = np.ascontiguousarray(state)  # Remove negative strides
    state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
    return state

    # Load pretrained networks if necessary
policy_net_weights_path = '/content/policy_net_weights.pth'
target_net_weights_path = '/content/policy_net_weights.pth'

policy_net.load_state_dict(torch.load(policy_net_weights_path))
target_net.load_state_dict(torch.load(target_net_weights_path))

target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
optimizer = optim.Adam(policy_net.parameters(), lr=learning_rate)

<ipython-input-7-0de6e5a10367>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  policy_net.load_state_dict(torch.load(policy_net_weights_path))
<ipython-input-7-0de6e5a1036

In [ ]:
import os

# Initialize log file

file_path = "training_log_dueling_14.txt"
if os.path.exists(file_path):
    os.remove(file_path)

with open(file_path, 'a') as file:
    file.write("Episode,Total_Reward,Moving_Average,Episode_Length,Step_Count,Success\n")


Training loop

In [ ]:
for episode in range(start_episode, num_episodes):
    # --- Training loop ---
    state, _ = env.reset()
    state = preprocess_state(state)
    done = False
    total_reward = 0
    frame_count = 0
    episode_length = 0
    x_pos_last = 40
    time_last = 400
    #standing_still_counter = 0

    while not done:
        action = select_action(state, epsilon)
        if episode > start_learning:
            epsilon = max(epsilon * epsilon_decay, epsilon_min)

        next_state, reward, terminated, truncated, next_info = env.step(action)
        next_state = preprocess_state(next_state)
        done = terminated or truncated

        total_reward += reward

        replayBuffer.append((state, action, reward, next_state, done))

        state = next_state

        if len(replayBuffer) > batch_size and episode > start_learning:
            batch = random.sample(replayBuffer, batch_size)
            states, actions, rewards, next_states, dones = zip(*batch)

            # Tensor conversion for training
            states = torch.cat(states)
            actions = torch.tensor(actions, dtype=torch.int64).to(device)
            rewards = torch.tensor(rewards).to(device)
            next_states = torch.cat(next_states)
            dones = torch.tensor(dones, dtype=torch.float32).to(device)

            # Compute Q-values and loss
            current_q_values = policy_net(states).gather(1, actions.unsqueeze(1)).squeeze(1)
            next_state_actions = policy_net(next_states).argmax(dim=1)
            next_q_values = target_net(next_states).gather(1, next_state_actions.unsqueeze(1)).squeeze(1)
            expected_q_values = rewards + gamma * next_q_values * (1 - dones)

            loss = loss_fn(current_q_values, expected_q_values.detach())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        step_count += 1
        episode_length += 1


        if done:
            # Save training rewards and calculate moving averages
            training_rewards.append(total_reward)
            moving_average_training.append(np.mean(training_rewards[-100:]))
            min_training_rewards.append(np.min(training_rewards[-100:]))
            max_training_rewards.append(np.max(training_rewards[-100:]))
            min_moving_average_training.append(np.min(moving_average_training[-100:]))
            max_moving_average_training.append(np.max(moving_average_training[-100:]))

            # Check if Mario completed the level
            if next_info['flag_get']:
                success = 1  # Mario successfully completed the level
            else:
                success = 0  # Mario did not complete the level

            # Output information about the episode
            print(f"Episode: {episode}, Total Reward: {total_reward}, Epsilon: {epsilon}")
            print(f"Replay Buffer Memory: {len(replayBuffer)}, Episode Length: {episode_length}, Step Count: {step_count}")
            print(f"Moving Average (Training): {moving_average_training[-1]}, Success: {success}")

            # Write episode data to log file
            with open(file_path, 'a') as file:
                file.write(f"{episode},{total_reward},{moving_average_training[-1]},{episode_length},{step_count},{success}\n")

    if step_count % 7500 == 0:
        target_net.load_state_dict(policy_net.state_dict())

      # --- Save model weights ---
    if episode % save_weights == 0:
        policy_weight_filename = f'policy_net_weights_{episode}.pth'
        target_weight_filename = f'target_net_weights_{episode}.pth'
        torch.save(policy_net.state_dict(), policy_weight_filename)
        torch.save(target_net.state_dict(), target_weight_filename)

# Save weights after training
torch.save(policy_net.state_dict(), 'policy_net_weights.pth')
torch.save(target_net.state_dict(), 'target_net_weights.pth')

files.download('policy_net_weights.pth')
files.download('target_net_weights.pth')

# Save rewards and moving averages to files for later analysis
np.save('training_rewards.npy', training_rewards)
#np.save('evaluation_rewards.npy', evaluation_rewards)
np.save('moving_average_training.npy', moving_average_training)
#np.save('moving_average_evaluation.npy', moving_average_evaluation)

files.download('training_rewards.npy')
files.download('moving_average_training.npy')

# Plotting Moving Averages with min and max
episodes = range(len(moving_average_training))

# Plot the result
plt.figure(figsize=(15, 5))

# Plot smoothed Training Moving Average with sliding min and max
plt.plot(episodes, moving_average_training, 'b-', label='Avg (Last 100 Episodes)', linewidth=2)
plt.plot(episodes, min_moving_average_training, 'r-', linewidth=1.5, label='Min (Last 100 episodes)')
plt.plot(episodes, max_moving_average_training, 'g-', linewidth=1.5, label='Max (Last 100 episodes)')

# Fill the area between min and max with lighter transparency
plt.fill_between(episodes, min_moving_average_training, max_moving_average_training, facecolor='blue', alpha=0.07)

# Titles and labels
plt.title('Dueling DQN Moving Average Rewards (Training)')
plt.xlabel('Episodes')
plt.ylabel('Reward Value')
plt.legend(loc='upper left')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()

env.close()


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
Replay Buffer Memory: 300000, Episode Length: 881, Step Count: 3542923
Moving Average (Training): 1595.68, Success: 1
Episode: 39706, Total Reward: 2188.0, Epsilon: 0.02
Replay Buffer Memory: 300000, Episode Length: 201, Step Count: 3543124
Moving Average (Training): 1605.86, Success: 1
Episode: 39707, Total Reward: 2165.0, Epsilon: 0.02
Replay Buffer Memory: 300000, Episode Length: 201, Step Count: 3543325
Moving Average (Training): 1605.64, Success: 1
Episode: 39708, Total Reward: 2129.0, Epsilon: 0.02
Replay Buffer Memory: 300000, Episode Length: 189, Step Count: 3543514
Moving Average (Training): 1605.1, Success: 0
Episode: 39709, Total Reward: 1144.0, Epsilon: 0.02
Replay Buffer Memory: 300000, Episode Length: 108, Step Count: 3543622
Moving Average (Training): 1594.69, Success: 0
Episode: 39710, Total Reward: 2187.0, Epsilon: 0.02
Replay Buffer Memory: 300000, Episode Length: 202, Step Count: 3543824
Moving Averag

KeyboardInterrupt: 